# Creación de dataframe combinando los datos disponibles
En este notebook vamos a hacer primer análisis del fichero convocatorias, disponibles en https://github.com/JaimeObregon/subvenciones/tree/main/files

Parte de este fichero está inspirado en otros contenidos generados desde que se liberó este dataset. En concreto, de las sigueintes referencias:
- https://nbviewer.org/github/astrojuanlu/subvenciones/blob/explorador/explorador/Exploracion%20convocatorias.ipynb 
- https://www.kaggle.com/code/elperitoinformtico/an-lisis-subvenciones-spain/
- https://github.com/dacamposol/subvenciones/blob/main/notebooks/load.ipynb

También ha resultado muy interesante la interacción en el canal de Discord que se ha lanzado!

Nota: Si estás en Linux/MacOS te tocará cambiar las rutas para cargar los ficheros de \ a /

## Objetivo
El objetivo de este cuaderno será genera un dataframe que contenga los datos combinados de los ficheros de convocatorias y las entidades jurídicas beneficiarias, para un analisis inicial.

In [28]:
import pandas as pd
import datetime as dt

# Si estás usando Linux o Mac, aquí es donde tienes que cambiar los separadores de directorio para que te funcione
# Formato Mac - Descomenta estas lineas
# juridicas_1_file = '../files/juridicas_1.csv.gz'
# juridicas_2_file = '../files/juridicas_2.csv.gz' 
#
# Formato Windows
convocatorias_file = '..\\files\\convocatorias.csv.gz'
juridicas_1_file = '..\\files\\juridicas_1.csv.gz'
juridicas_2_file = '..\\files\\juridicas_2.csv.gz'


Como ya hemos trabajado los ficheros con anterioridad, vamos a importarlos directamente. Preparamos primero los encabezados de ambos ficheros.

In [29]:
nombres_columnas_convocatorias = [
  "IDConv", #0
  "id", #1
  "mrr", #2
  "convocanteN1", #3
  "convocanteN2", #4
  "convocanteN3", #5
  "fechareg", #6
  "titulo", #7
  "bbreguladoras", #8 - nombre tomado de los ficheros de jurídicas
  "tituloleng", #9
  "verConcesiones", #10 - valores nulos
  "dummy1", #11 - valor 350078 fijo
  "dummy2"  #12 - valor igual a IDConv +1
]

etiquetas_columnas_convocatorias = [
  "IDConv",
  "Código BDNS",
  "MRR",
  "Administración",
  "Departamento",
  "Órgano",
  "Fecha de registro",
  "Título de la convocatoria",
  "URL de las BBRR",
  "Título cooficial",
  "Ver concesiones",
  "dummy1",
  "dummy2",
]

# Eliminamos los tres últimos elementos de la lista de nombres_columnas
columnas_utiles_convocatorias = nombres_columnas_convocatorias[: -3]

print(columnas_utiles_convocatorias)

['IDConv', 'id', 'mrr', 'convocanteN1', 'convocanteN2', 'convocanteN3', 'fechareg', 'titulo', 'bbreguladoras', 'tituloleng']


In [30]:
nombres_columnas_juridicas = [
  "ID", #0
  "IDConv", #1
  "convocanteN1", #2
  "convocanteN2", #3
  "convocanteN3", #4
  "convocatoria", #5
  "bbreguladoras", #6
  "programa", #7
  "fechaconc", #8
  "beneficiario", #9
  "importe", #10
  "instrumento", #11
  "ayudaequiv", #12
  "detalles", #13
  "proyecto", #14
  "sancion", #15
  "numcov" #16

]
etiquetas_columnas_juridicas = [
  "ID",
  "IDConvocatoria",
  "Administración",
  "Departamento",
  "Órgano",
  "URL de las BBRR",
  "Aplicación presupuestaria",
  "Fecha de concesión",
  "Beneficiario",
  "Importe",
  "Instrumento",
  "Ayuda Equivalente",
  "Detalles",
  "proyecto",
  "sancion",
  "numcov"
]

**Nota: El ID de convocatorias es el código BDNS de la subvención. El ID de jurídicas es el identificador de la concesión de la subvención. El campo que si es común entre ambos datasets es IDConv (ID de la convocatoria)**

Importamos ahora cada fichero en un dataframe.

In [31]:
# Convocatorias
convocatorias = pd.read_csv(convocatorias_file,
names = nombres_columnas_convocatorias,
index_col="IDConv",
dtype={
    "IDConv": object,
    "detalles": object,
},
usecols=columnas_utiles_convocatorias,
parse_dates=["fechareg"],
date_parser=lambda d: dt.datetime.strptime(d, "%d/%m/%Y"),
).sort_values("fechareg")
convocatorias.head()


C:\Users\Mi PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,mrr,convocanteN1,convocanteN2,convocanteN3,fechareg,titulo,bbreguladoras,tituloleng
IDConv,,,,,,,,,
49114,200820085002270F,NO,CANTABRIA,SERVICIO CÁNTABRO DE EMPLEO,NaN,2008-06-24,CENTROS ESPECIALES DE EMPLEO: Proyectos Genera...,Información no disponible en BDNS2007,NaN
34230,200820075014000F,NO,CANTABRIA,DIRECCION DEL ICASS,NaN,2008-08-22,PROMOCION SOCIAL SINDICAL,Información no disponible en BDNS2007,NaN
49122,200820085002350F,NO,CANTABRIA,SERVICIO CÁNTABRO DE EMPLEO,NaN,2008-10-08,"Decreto 33/2008, de 3 de abril, por el que se ...",Información no disponible en BDNS2007,NaN
49121,200820085002340F,NO,CANTABRIA,SERVICIO CÁNTABRO DE EMPLEO,NaN,2008-10-08,"Decreto 34/2008, de 3 de abril, por el que se ...",Información no disponible en BDNS2007,NaN
32279,2007200763520703,NO,PONFERRADA,AYUNTAMIENTO DE PONFERRADA,NaN,2009-05-01,CONVOCATORIA SUBVENCIONES A ASOCIACIONES PARA ...,Información no disponible en BDNS2007,NaN


In [32]:
convocatorias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350078 entries, 49114 to 819060
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             350078 non-null  object        
 1   mrr            350078 non-null  object        
 2   convocanteN1   350078 non-null  object        
 3   convocanteN2   350078 non-null  object        
 4   convocanteN3   54063 non-null   object        
 5   fechareg       350078 non-null  datetime64[ns]
 6   titulo         350078 non-null  object        
 7   bbreguladoras  350078 non-null  object        
 8   tituloleng     17953 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 26.7+ MB


In [33]:
# Beneficiarias
juridicas_1 = pd.read_csv(juridicas_1_file,
names = nombres_columnas_juridicas,
index_col="ID",
dtype={
    "IDConv": str,
    "detalles": object,
},
parse_dates=["fechaconc"],
date_parser=lambda d: dt.datetime.strptime(d, "%d/%m/%Y"),
).sort_values("fechaconc")

juridicas_2 = pd.read_csv(juridicas_1_file,
names = nombres_columnas_juridicas,
index_col="ID",
dtype={
    "IDConv": str,
    "detalles": object,
},
parse_dates=["fechaconc"],
date_parser=lambda d: dt.datetime.strptime(d, "%d/%m/%Y"),
).sort_values("fechaconc")

beneficiarias = pd.concat([juridicas_1,juridicas_2])
beneficiarias.head()


,IDConv,convocanteN1,convocanteN2,convocanteN3,convocatoria,bbreguladoras,programa,fechaconc,beneficiario,importe,instrumento,ayudaequiv,detalles,proyecto,sancion,numcov
ID,,,,,,,,,,,,,,,,
38960284,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,2019-09-24,G16276230 ASOC. DESARROLLO INTEGRAL DE ZARZUELA,278.52,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,278.52,0,455504,10435022,9427075
38643060,647651,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,P4309900A NOU DE GAIA LA AJUNTAMENT,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425958
38643061,647651,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,P4314300G AJUNTAMENT DE PONTILS,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425959
38643062,647651,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,"P4309700E MORERA DE MONTSANT LA, AJUNTAMENT",11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425960
38643063,647651,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,P4311900G PRAT DE COMTE AJUNTAMENT,9589.33,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,9589.33,0,446091,10435022,9425961


In [34]:
beneficiarias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3991670 entries, 38960284 to 65045185
Data columns (total 16 columns):
 #   Column         Dtype         
---  ------         -----         
 0   IDConv         object        
 1   convocanteN1   object        
 2   convocanteN2   object        
 3   convocanteN3   object        
 4   convocatoria   object        
 5   bbreguladoras  object        
 6   programa       object        
 7   fechaconc      datetime64[ns]
 8   beneficiario   object        
 9   importe        float64       
 10  instrumento    object        
 11  ayudaequiv     float64       
 12  detalles       object        
 13  proyecto       object        
 14  sancion        int64         
 15  numcov         int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(11)
memory usage: 517.7+ MB


El fichero beneficiarios tiene índices repetidos, que vienen del fichero juridicas_1. Mientras averiguamos la causa, vamos a eliminar las filas duplicadas.

In [35]:
beneficiarias.shape

(3991670, 16)

In [36]:
beneficiarias.index.has_duplicates


True

In [37]:
idx = beneficiarias.index
df_dup = beneficiarias[idx.duplicated()]
df_dup.shape

(2011729, 16)

In [38]:
beneficiarias = beneficiarias[~beneficiarias.index.duplicated(keep='first')]

In [39]:
beneficiarias.shape

(1979941, 16)

In [40]:
beneficiarias.index.has_duplicates

False

# Uniendo ambos datasets
Antes de unir los datasets vamos a realizar una consulta a ambos datasets, para ver los campos de unión necesarios.

In [41]:
# Convocatorias - IDConv está configurado como índice
convocatorias.loc[657064]

id                                                          455504
mrr                                                             NO
convocanteN1                            DIPUTACIÓN PROV. DE CUENCA
convocanteN2                       DIPUTACIÓN PROVINCIAL DE CUENCA
convocanteN3                                                   NaN
fechareg                                       2019-05-16 00:00:00
titulo                           CT. CONVOCATORIA CUENCA PARTICIPA
bbreguladoras    https://www.dipucuenca.es/documents/12423/5606...
tituloleng                                                     NaN
Name: 657064, dtype: object

In [42]:
# Beneficiarias - IDConv es una columna
beneficiarias.loc[beneficiarias['IDConv'] == '657064']

,IDConv,convocanteN1,convocanteN2,convocanteN3,convocatoria,bbreguladoras,programa,fechaconc,beneficiario,importe,instrumento,ayudaequiv,detalles,proyecto,sancion,numcov
ID,,,,,,,,,,,,,,,,
38960284,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,2019-09-24,G16276230 ASOC. DESARROLLO INTEGRAL DE ZARZUELA,278.52,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,278.52,0,455504,10435022,9427075
38960283,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,2019-09-24,G16338881 ASOC DEFENSA DE ECON RESPONS ALMENDROS,309.80,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,309.80,0,455504,10435022,9427074
38775552,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,2019-09-24,G16151128 A.P.A. SAN ANTONIO DE TALAYUELAS,730.93,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,730.93,0,455504,10435022,9427001
38775556,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,2019-09-24,G16275133 AS. JUVENIL LA ERMITILLA,227.00,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,227.00,0,455504,10435022,9427002
38775557,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,2019-09-24,J16227258 A. C. LA PEÑUELA LA HINOJOSA,289.10,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,289.10,0,455504,10435022,9427003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38757962,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2019-305/334/4800012,2019-09-24,G16308983 ASOCIACION CULTURAL RECREATIVA CAÑAV...,397.20,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,397.20,0,455504,10435022,9426315
38757963,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2019-305/334/4800012,2019-09-24,G16317893 ASOCIACION DE MUJERES FLOR DE AZAFRAN,187.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,187.50,0,455504,10435022,9426316
38757975,657064,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2019-305/334/4800012,2019-09-24,G16290371 AS. SOCIO-CULTURAL AMIGOS DEL PARQUE...,243.10,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,243.10,0,455504,10435022,9426317


In [43]:
convocatorias.index.inferred_type


'integer'

In [44]:
convocatorias.index

Int64Index([ 49114,  34230,  49122,  49121,  32279,  30921,  53336,  32387,
             54421,  99512,
            ...
            818906, 818909, 818911, 818912, 818913, 818915, 818916, 818917,
            818908, 819060],
           dtype='int64', name='IDConv', length=350078)

In [45]:
beneficiarias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1979941 entries, 38960284 to 65045185
Data columns (total 16 columns):
 #   Column         Dtype         
---  ------         -----         
 0   IDConv         object        
 1   convocanteN1   object        
 2   convocanteN2   object        
 3   convocanteN3   object        
 4   convocatoria   object        
 5   bbreguladoras  object        
 6   programa       object        
 7   fechaconc      datetime64[ns]
 8   beneficiario   object        
 9   importe        float64       
 10  instrumento    object        
 11  ayudaequiv     float64       
 12  detalles       object        
 13  proyecto       object        
 14  sancion        int64         
 15  numcov         int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(11)
memory usage: 321.3+ MB


In [46]:
beneficiarias['IDConv'] = beneficiarias['IDConv'].astype('int64')
assert beneficiarias['IDConv'].dtype == 'int64'

In [47]:
conv_benef = convocatorias.merge(beneficiarias, how='right',suffixes=('_conv','_benef'), left_index=True, right_on = 'IDConv')

## Verificando la salida

In [48]:
conv_benef.head()

,id,mrr,convocanteN1_conv,convocanteN2_conv,convocanteN3_conv,fechareg,titulo,bbreguladoras_conv,tituloleng,IDConv,...,programa,fechaconc,beneficiario,importe,instrumento,ayudaequiv,detalles,proyecto,sancion,numcov
ID,,,,,,,,,,,,,,,,,,,,,
38960284,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2020-305/334/4800012,2019-09-24,G16276230 ASOC. DESARROLLO INTEGRAL DE ZARZUELA,278.52,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,278.52,0,455504,10435022,9427075
38643060,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,P4309900A NOU DE GAIA LA AJUNTAMENT,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425958
38643061,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,P4314300G AJUNTAMENT DE PONTILS,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425959
38643062,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,"P4309700E MORERA DE MONTSANT LA, AJUNTAMENT",11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425960
38643063,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019-09-24,P4311900G PRAT DE COMTE AJUNTAMENT,9589.33,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,9589.33,0,446091,10435022,9425961


In [49]:
conv_benef.loc[conv_benef['IDConv'] == 657064]

,id,mrr,convocanteN1_conv,convocanteN2_conv,convocanteN3_conv,fechareg,titulo,bbreguladoras_conv,tituloleng,IDConv,...,programa,fechaconc,beneficiario,importe,instrumento,ayudaequiv,detalles,proyecto,sancion,numcov
ID,,,,,,,,,,,,,,,,,,,,,
38960284,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2020-305/334/4800012,2019-09-24,G16276230 ASOC. DESARROLLO INTEGRAL DE ZARZUELA,278.52,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,278.52,0,455504,10435022,9427075
38960283,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2020-305/334/4800012,2019-09-24,G16338881 ASOC DEFENSA DE ECON RESPONS ALMENDROS,309.80,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,309.80,0,455504,10435022,9427074
38775552,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2020-305/334/4800012,2019-09-24,G16151128 A.P.A. SAN ANTONIO DE TALAYUELAS,730.93,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,730.93,0,455504,10435022,9427001
38775556,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2020-305/334/4800012,2019-09-24,G16275133 AS. JUVENIL LA ERMITILLA,227.00,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,227.00,0,455504,10435022,9427002
38775557,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2020-305/334/4800012,2019-09-24,J16227258 A. C. LA PEÑUELA LA HINOJOSA,289.10,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,289.10,0,455504,10435022,9427003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38757962,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2019-305/334/4800012,2019-09-24,G16308983 ASOCIACION CULTURAL RECREATIVA CAÑAV...,397.20,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,397.20,0,455504,10435022,9426315
38757963,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2019-305/334/4800012,2019-09-24,G16317893 ASOCIACION DE MUJERES FLOR DE AZAFRAN,187.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,187.50,0,455504,10435022,9426316
38757975,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2019-305/334/4800012,2019-09-24,G16290371 AS. SOCIO-CULTURAL AMIGOS DEL PARQUE...,243.10,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,243.10,0,455504,10435022,9426317


# Reorganizando y limpiando los datos mezclados

Separamos el CIF en una columna independiente

In [50]:
conv_benef[['cif_beneficiario','beneficiario']] = conv_benef['beneficiario'].str.split(" ", n=1, expand=True)

In [51]:
conv_benef.head()

,id,mrr,convocanteN1_conv,convocanteN2_conv,convocanteN3_conv,fechareg,titulo,bbreguladoras_conv,tituloleng,IDConv,...,fechaconc,beneficiario,importe,instrumento,ayudaequiv,detalles,proyecto,sancion,numcov,cif_beneficiario
ID,,,,,,,,,,,,,,,,,,,,,
38960284,455504,NO,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,2019-05-16,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,NaN,657064,...,2019-09-24,ASOC. DESARROLLO INTEGRAL DE ZARZUELA,278.52,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,278.52,0,455504,10435022,9427075,G16276230
38643060,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-09-24,NOU DE GAIA LA AJUNTAMENT,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425958,P4309900A
38643061,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-09-24,AJUNTAMENT DE PONTILS,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425959,P4314300G
38643062,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-09-24,"MORERA DE MONTSANT LA, AJUNTAMENT",11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,446091,10435022,9425960,P4309700E
38643063,446091,NO,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019-03-25,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019 - Subvencions PAES,647651,...,2019-09-24,PRAT DE COMTE AJUNTAMENT,9589.33,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,9589.33,0,446091,10435022,9425961,P4311900G


La columna id y la columna proyecto son iguales. Eliminamos la columna id y dejamos la otra que es más explicativa. También eliminamos el resto de columnas duplicadas.

A continuación renombraremos las columnas, moveremos IDConv al comienzo del dataframe y el cif, junto al beneficiario.

In [52]:
conv_benef.drop(columns=['id', 'convocatoria', 'convocanteN1_benef', 'convocanteN2_benef', 'convocanteN3_benef','bbreguladoras_benef'], inplace=True)
conv_benef.rename(columns={'convocanteN1_conv':'convocanteN1', 'convocanteN2_conv':'convocanteN2', 'convocanteN3_conv':'convocanteN3', 'bbreguladoras_conv':'bbreguladoras'}, inplace=True)
conv_benef.columns

Index(['mrr', 'convocanteN1', 'convocanteN2', 'convocanteN3', 'fechareg',
       'titulo', 'bbreguladoras', 'tituloleng', 'IDConv', 'programa',
       'fechaconc', 'beneficiario', 'importe', 'instrumento', 'ayudaequiv',
       'detalles', 'proyecto', 'sancion', 'numcov', 'cif_beneficiario'],
      dtype='object')

In [53]:
conv_benef = conv_benef[['IDConv', 'fechareg', 'convocanteN1', 'convocanteN2', 'convocanteN3', 
       'titulo', 'bbreguladoras', 'programa', 'tituloleng' , 
        'fechaconc', 'cif_beneficiario', 'beneficiario', 'importe', 'instrumento',
       'ayudaequiv', 'detalles', 'mrr' ,'proyecto', 'sancion', 'numcov',
       ]]

# Resultado final

In [54]:
conv_benef.head()

,IDConv,fechareg,convocanteN1,convocanteN2,convocanteN3,titulo,bbreguladoras,programa,tituloleng,fechaconc,cif_beneficiario,beneficiario,importe,instrumento,ayudaequiv,detalles,mrr,proyecto,sancion,numcov
ID,,,,,,,,,,,,,,,,,,,,
38960284,657064,2019-05-16,DIPUTACIÓN PROV. DE CUENCA,DIPUTACIÓN PROVINCIAL DE CUENCA,NaN,CT. CONVOCATORIA CUENCA PARTICIPA,https://www.dipucuenca.es/documents/12423/5606...,2020-305/334/4800012,NaN,2019-09-24,G16276230,ASOC. DESARROLLO INTEGRAL DE ZARZUELA,278.52,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,278.52,0,NO,455504,10435022,9427075
38643060,647651,2019-03-25,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019 - Subvencions PAES,2019-09-24,P4309900A,NOU DE GAIA LA AJUNTAMENT,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,NO,446091,10435022,9425958
38643061,647651,2019-03-25,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019 - Subvencions PAES,2019-09-24,P4314300G,AJUNTAMENT DE PONTILS,11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,NO,446091,10435022,9425959
38643062,647651,2019-03-25,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019 - Subvencions PAES,2019-09-24,P4309700E,"MORERA DE MONTSANT LA, AJUNTAMENT",11632.50,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,11632.50,0,NO,446091,10435022,9425960
38643063,647651,2019-03-25,DIPUTACIÓN PROV. DE TARRAGONA,DIPUTACIÓN PROVINCIAL DE TARRAGONA,NaN,2019 Subvenciones para la implantación de mej...,https://www.diputaciodetarragona.cat/ebop/inde...,2019-2019-1200-425-76200-01-SUBVENCIONS AJUNTA...,2019 - Subvencions PAES,2019-09-24,P4311900G,PRAT DE COMTE AJUNTAMENT,9589.33,SUBVENCIÓN Y ENTREGA DINERARIA SIN CONTRAPREST...,9589.33,0,NO,446091,10435022,9425961


¡A disfrutarlo!